In [9]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from hurst import compute_Hc


In [2]:
db_connection_str = 'mysql+pymysql://student:SoQ_trading24/25@3.81.230.185/trading'
db_connection = create_engine(db_connection_str)

In [3]:
query = 'SELECT * from trading.futures_1min_bar'
df = pd.read_sql(query, con=db_connection)
print(df.shape)
df.head()

(312823, 8)


,contract,instrument,bid,offer,bid_size,offer_size,timestamp,adjustment
0,ES,ESZ3,4572.75,4573.00,21,31,2023-12-01 00:01:00,316.375
1,ES,ESZ3,4572.50,4572.75,31,23,2023-12-01 00:02:00,316.375
2,ES,ESZ3,4571.75,4572.00,69,20,2023-12-01 00:03:00,316.375
3,ES,ESZ3,4571.75,4572.00,67,21,2023-12-01 00:04:00,316.375
4,ES,ESZ3,4571.75,4572.00,24,47,2023-12-01 00:05:00,316.375


In [6]:
df.to_parquet('data/futures_1min_bar.parquet')

In [ ]:
fx_instrument_sql = "SELECT distinct instrument from trading.fx_1min_bar"
df_fx_instrument = pd.read_sql(fx_instrument_sql, con=db_connection)
fx_list = [fx.replace('"', '') for fx in df_fx_instrument.instrument.values]

In [ ]:
for fx in tqdm(fx_list):
    fx_sql = f"SELECT * from trading.fx_1min_bar where instrument = '\"{fx}\"'"
    df_fx = pd.read_sql(fx_sql, con=db_connection)

    df_fx.to_parquet(f'data/{fx}_data.parquet')

In [4]:
df.groupby(['contract', 'instrument']).agg({'timestamp':['min', 'max']}).reset_index()\
                                                .sort_values(by=('timestamp', 'min'))

contract instrument           timestamp                    
                                      min                 max
4       ES       ESZ3 2023-12-01 00:01:00 2023-12-11 21:59:00
0       ES       ESH4 2023-12-11 22:00:00 2024-03-11 21:00:00
2       ES       ESM4 2024-03-11 21:00:00 2024-06-17 20:57:00
3       ES       ESU4 2024-06-17 21:01:00 2024-09-16 20:59:00
5       ES       ESZ4 2024-09-16 21:00:00 2024-12-16 22:00:00
1       ES       ESH5 2024-12-16 22:00:00 2025-02-24 17:32:00

In [38]:
df = df.sort_values(by='timestamp')
df = df[~df.bid.isna()]
df['Roll Adjusted Bid'] = df['bid'] + df['adjustment']
df['Roll Adjusted Offer'] = df['offer'] + df['adjustment']


df['mid_price'] = (df['Roll Adjusted Bid'] + df['Roll Adjusted Offer']) / 2